In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [2]:
prototxtPath=os.path.sep.join(['deploy.prototxt'])
weightsPath=os.path.sep.join(['res10_300x300_ssd_iter_140000.caffemodel'])

In [3]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

In [4]:
model=load_model('mobilenet_v2.model')

In [44]:
image=cv2.imread('face-mask-study.jpg')

In [45]:
(h,w)=image.shape[:2]

In [46]:
blob=cv2.dnn.blobFromImage(image,1.0,(300,300),(104.0,177.0,123.0))

In [47]:
blob.shape

(1, 3, 300, 300)

In [48]:
net.setInput(blob)
detections=net.forward()

In [49]:
detections

array([[[[0.        , 1.        , 0.9999907 , ..., 0.58717924,
          0.36393982, 0.9132052 ],
         [0.        , 1.        , 0.99980253, ..., 0.588965  ,
          0.7752143 , 0.931941  ],
         [0.        , 1.        , 0.9990114 , ..., 0.07609312,
          0.7842918 , 0.44108248],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [51]:
#looping over the detections
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #bounding boxes
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extracting region of indentification and preprocessing
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(224,224))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        
        (mask,withoutMask)=model.predict(face)[0]
        
        #class lables and binding boxes
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #adding lables
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()